In [1]:
!pip install pandas matplotlib numpy nltk sentence_transformers scikit-learn

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
import ssl
import ast

nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
#Text processing
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import string

from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm
tqdm.pandas()

engStops = stopwords.words('english')
ps = PorterStemmer()
lt = WordNetLemmatizer()

In [4]:
credits = pd.read_csv('/content/credits.csv')
movies_metadata = pd.read_csv('/content/movies_metadata.csv')
keywords = pd.read_csv('/content/keywords.csv')

/tmp/ipython-input-1939864020.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('/content/movies_metadata.csv')


In [5]:
merged_data = credits.merge(keywords, on='id', how='inner')

In [6]:
movies_metadata.drop(19730, inplace=True)
movies_metadata.drop(29503, inplace=True)
movies_metadata.drop(35587, inplace=True)

In [7]:
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [8]:
movies_metadata['id'] = movies_metadata['id'].apply(lambda x: int(x))

In [9]:
merged_data = merged_data.merge(movies_metadata, on='id', how='inner')

In [10]:
merged_data.shape

(46628, 27)

In [11]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46628 entries, 0 to 46627
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cast                   46628 non-null  object 
 1   crew                   46628 non-null  object 
 2   id                     46628 non-null  int64  
 3   keywords               46628 non-null  object 
 4   adult                  46628 non-null  object 
 5   belongs_to_collection  4574 non-null   object 
 6   budget                 46628 non-null  object 
 7   genres                 46628 non-null  object 
 8   homepage               8009 non-null   object 
 9   imdb_id                46611 non-null  object 
 10  original_language      46617 non-null  object 
 11  original_title         46628 non-null  object 
 12  overview               45633 non-null  object 
 13  popularity             46624 non-null  object 
 14  poster_path            46229 non-null  object 
 15  pr

In [12]:
data = merged_data
data.isna().sum()

,0
cast,0
crew,0
id,0
keywords,0
adult,0
belongs_to_collection,42054
budget,0
genres,0
homepage,38619
imdb_id,17


In [13]:
data.drop(columns=['belongs_to_collection','budget', 'homepage', 'popularity', 'poster_path', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'video'], inplace=True)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46628 entries, 0 to 46627
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cast                  46628 non-null  object 
 1   crew                  46628 non-null  object 
 2   id                    46628 non-null  int64  
 3   keywords              46628 non-null  object 
 4   adult                 46628 non-null  object 
 5   genres                46628 non-null  object 
 6   imdb_id               46611 non-null  object 
 7   original_language     46617 non-null  object 
 8   original_title        46628 non-null  object 
 9   overview              45633 non-null  object 
 10  production_companies  46624 non-null  object 
 11  production_countries  46624 non-null  object 
 12  spoken_languages      46624 non-null  object 
 13  vote_average          46624 non-null  float64
 14  vote_count            46624 non-null  float64
dtypes: float64(2), int6

In [15]:
data.isna().sum()

,0
cast,0
crew,0
id,0
keywords,0
adult,0
genres,0
imdb_id,17
original_language,11
original_title,0
overview,995


In [16]:

def cast_pp(txt):
  try:
    txt = ast.literal_eval(txt)
  except:
    return ''

  casts = [member.get('name', '') for member in txt[:3] if 'name' in member]
  fcast = ['_'.join(word_tokenize(cast)).lower() for cast in casts if cast]

  return ' '.join(fcast)

In [17]:
data['cast'] = data['cast'].progress_apply(cast_pp)

100%|██████████| 46628/46628 [00:31<00:00, 1476.06it/s]


In [18]:
def crew_pp(txt):
  try:
    txt = ast.literal_eval(txt)
    director = ''
    for t in txt:
      if t['department'].lower() == 'directing' or t['department'].lower() == 'director':
        director = t['name']
        break
    director = word_tokenize(director.lower())
    director = '_'.join(director)
    return director

  except:
    return ''


In [19]:
data['crew'] = data['crew'].progress_apply(crew_pp)

100%|██████████| 46628/46628 [00:18<00:00, 2539.76it/s]


In [20]:
def keywords_pp(txt):
  txt = ast.literal_eval(txt)
  keywords = [member.get('name', '') for member in txt if member]
  keywords = [''.join([letter for letter in keyword if letter not in string.punctuation]) for keyword in keywords]
  keywords = ['_'.join(word_tokenize(keyword)).lower() for keyword in keywords]
  return "".join(keywords)

In [21]:
data['keywords'] = data['keywords'].progress_apply(keywords_pp)

100%|██████████| 46628/46628 [00:06<00:00, 7672.80it/s]


In [22]:
def getNamefromDict(txt):
  txt = ast.literal_eval(txt)
  genres = [member.get('name', '') for member in txt if member]
  genres = ['_'.join(word_tokenize(genre)).lower() for genre in genres if genre]
  return " ".join(genres)

In [23]:
data['genres'] = data['genres'].progress_apply(getNamefromDict)

100%|██████████| 46628/46628 [00:04<00:00, 10362.42it/s]


In [24]:
def overview_pp(txt):
  try:
    bans = ['--']
    txt = [word.lower() for word in txt if word not in string.punctuation]
    txt = ''.join(txt)
    txt = word_tokenize(txt)
    txt = [word for word in txt if word not in bans]
    txt = [lt.lemmatize(ps.stem(word)) for word in txt if word not in engStops]
    return ' '.join(txt)
  except:
    return ''


In [25]:
data['overview'] = data['overview'].progress_apply(overview_pp)

100%|██████████| 46628/46628 [00:25<00:00, 1815.18it/s]


In [26]:
def getFirstFromDict(txt):
  try:
    txt = ast.literal_eval(txt)
    txt = txt[0]['name'].lower()
    txt = word_tokenize(txt)
    txt = [word for word in txt if word not in string.punctuation]
    txt = '_'.join(txt)
    return txt
  except:
    return ''

In [27]:
data.production_companies = data.production_companies.progress_apply(getFirstFromDict)
data.production_countries = data.production_countries.progress_apply(getFirstFromDict)
data.spoken_languages = data.spoken_languages.progress_apply(getFirstFromDict)

100%|██████████| 46628/46628 [00:01<00:00, 24260.35it/s]


In [28]:
data['processed_title'] = data['original_title'].progress_apply(lambda x: '_'.join([word.lower() for word in word_tokenize(x) if word not in string.punctuation]))

100%|██████████| 46628/46628 [00:01<00:00, 29610.01it/s]


In [29]:
data['content_merged'] = (
    data['crew'].fillna('') + ' '
    + data['keywords'].fillna('') + ' '
    + data['genres'].fillna('') + ' '
    + data['original_language'].fillna('') + ' '
    + data['overview'].fillna('') + ' '
    + data['production_companies'].fillna('') + ' '
    + data['production_countries'].fillna('') + ' '
    + data['spoken_languages'].fillna('') + ' '
    + data['cast'].fillna('') + ' '
    + data['processed_title'].fillna(''))

In [30]:
model = SentenceTransformer('all-mpnet-base-v2').to('cuda')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
data['vectors'] = model.encode(data['content_merged'].tolist(), show_progress_bar=True).tolist()

Batches:   0%|          | 0/1458 [00:00<?, ?it/s]

In [32]:
vectors = np.stack(data['vectors'].values)

In [33]:
!pip install torch
import torch

In [34]:
# def gpu_cosine_similarity(vectors):
#   if isinstance(vectors, list):
#     vectors = np.array(vectors)

#   tensor = torch.tensor(vectors, device='cuda', dtype=torch.float32)

#   tensor_norm  = torch.nn.functional.normalize(tensor, p=2, dim=1)

#   similarity = torch.mm(tensor_norm, tensor_norm.T)

#   return similarity.cpu().numpy()

In [35]:
# similarity_matrix = gpu_cosine_similarity(vectors)

In [36]:
def recommend_movies(vectors, title, top_n=5):
  idx = data[data['original_title'] == title].index[0]
  # if not idx:
  #   return f"Film {title} not found"
  # idx = idx.index[0]
  target_vector = vectors[idx]

  similarities = np.dot(vectors, target_vector) / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(target_vector))

  similar_indices = similarities.argsort()[-(top_n+1):-1][::-1]
  return similar_indices, similarities[similar_indices]

In [37]:
# def recommend_movies(title, top_n=5):
#   idx = data[data['original_title'] == title].index[0]
#   similar_scores = list(enumerate(similarity_matrix[idx]))
#   similar_scores = sorted(similar_scores, key = lambda x: x[1], reverse=True)[1:top_n+1]
#   recommendations = [data.iloc[i[0]]['original_title'] for i in similar_scores]
#   return recommendations

In [55]:
similar_indices, scores = recommend_movies(vectors, 'Home Alone')

In [56]:
data['original_title'][similar_indices]

,original_title
273,Miracle on 34th Street
6838,Bad Santa
2322,National Lampoon's Christmas Vacation
2852,Home Alone 2: Lost in New York
13196,Four Christmases


In [51]:
type(similar_indices)

numpy.ndarray